In [166]:
#FINALIZED PYTHON VERSION

import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Galactocentric, ICRS, Galactic
import gala.coordinates as gc
    
#Constants
asec_to_rad = 206264.8 #Converts arcseconds to radians
sec_to_year = 3.155693e7 #Converts from years to seconds
pc_to_km = 3.08568e13 #Converts from pc to km
year_to_day = 365.2422 #Converts years to days
ra_lsr = 318.00425 #RA of the apex of the LSR motion (ie., l=90, b=0)     
dec_lsr = 48.329611 #DEC of the apex of the LSR motion (J2000.0)
ra_sa = 251.51 #RA of the apex of the solar peculiar motion (wrt LSR) J2000.0 
dec_sa = 10.129 #DEC of the apex of the solar peculiar motion (wrt LSR)
alpha_NGP = 192.85948 #RA in degrees of the Galactic Pole (J2000.0)
delta_NGP = 27.12825 #DEC in degrees of the Galactic Pole (J2000.0)
gc_GAN = 32.93192 #Angle along galactic equator from (l,b)=(0,0) to galactic ascending node
l_NCP = 122.9319#Galactic longitude of the NCP (J2000.0)
b_NCP =  27.1283#Galactic latitude of the NCP  (J2000.0)
usun = -10.0 #U-component of Sun's velocity w.r.t. LSR; Dehnen & Binney                  
vsun = 11.0 #V-component of Sun's velocity w.r.t. LSR; 1998, MNRAS, 
wsun = 7.0 #W-component of Sun's velocity w.r.t. LSR; 298, 387
v_lsr = 237.0 #Circular velocity of LSR w.r.t. Galaxy center in km/s
d_lsr = 8200.0 #Distance of LSR from the Galaxy center in pc

#Functions

def sin_deg(x):
    #Sine function for degree-valued args
    deg_to_rad = 1.74532925199e-02
    return np.sin(np.deg2rad(x))

def cos_deg(x):
    #Cosine function for degree-valued args
    return np.cos(np.deg2rad(x))

def asin_deg(x):
    #Gives asin in degrees
    return np.rad2deg(np.arcsin(x))

def acos_deg(x):
    #Gives acos in degrees
    return np.rad2deg(np.arccos(x))

def atan2_deg(y,x):
    #Gives atan2 in degrees
    return np.rad2deg(np.arctan2(y,x))

def radec_to_galactic_pos(ra, dec):    
    #Returns dictionary of galactic coords (l, b) in degrees, given an object's RA & DEC in degrees
    coords = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    galactic = coords.galactic
    return {'l':galactic.l.deg, 'b':galactic.b.deg}

radec_to_galactic_pos(test_ra, test_dec)

{'b': 67.23124435847123, 'l': 220.16913736951736}

In [163]:
def pm_radec_to_gal(ra, dec, dist, mu_a, mu_d, rv):
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.kpc)
    pm = [mu_a, mu_d] * u.uas/(100*u.yr) 
    return gc.pm_icrs_to_gal(c, pm).to(u.uas/(u.yr))

pm_radec_to_gal(test_ra, test_dec, test_d, mu_a, mu_d, rv)

<Quantity [ 0.06170815,-0.09225772] uas / yr>

## Development

In [121]:
test_ra = 168.3700 #degrees
test_dec = 22.15167 #degrees
test_d = 233000 #pc
mu_a = -6.930000 #mas/cent
mu_d = -8.670000 #mas/cent
rv = 79.1 #km/s

galcoords = radec_to_galactic_pos(test_ra, test_dec)
l = galcoords['l']
b = galcoords['b']
print l, b

220.16913737 67.2312443585


In [172]:
def getxyz(ra, dec, d):
    global b
    global l
    c = ICRS(ra=ra*u.degree, dec=dec*u.degree, distance=d*u.pc)
    return c.transform_to(Galactocentric(galcen_distance=8.2*u.kpc, z_sun=27*u.pc))
getxyz(test_ra, test_dec, test_d)

<Galactocentric Coordinate (galcen_distance=8.2 kpc, galcen_ra=266d24m18.36s, galcen_dec=-28d56m10.23s, z_sun=27.0 pc, roll=0.0 deg): (x, y, z) in pc
    (-76398.40831938, -58166.63682664,  215095.85734564)>

In [167]:
def velocity(ra, dec, dist, mu_a, mu_d, rv):
    global test_dec #????
    pm = [mu_a, mu_d] * u.uas/(100*u.yr) 
    rv = rv * u.km/u.s
    frame = Galactocentric(z_sun=27.*u.pc, galcen_distance=8.2*u.kpc)
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.kpc)
    return gc.vhel_to_gal(c.icrs, pm=pm, rv=rv, galactocentric_frame=frame, vlsr=[usun, vsun, wsun]*u.km/u.s)
    
velocity(test_ra, test_dec, test_d, mu_a, mu_d, rv)

<Quantity [-61.11616968, 98.56157507, 40.66716428] km / s>

In [100]:
def galactic_rect_pos(l, b, d, d_lsr):
    x0 = d_lsr-d*cos_deg(b)*cos_deg(l)
    y0 = -d*cos_deg(b)*sin_deg(l)
    z0 = d*sin_deg(b)
    x = x0/1000
    y = y0/1000
    z = z0/1000
    print x, y, z
galactic_rect_pos(l, b, test_d, d_lsr)

77.1058750111 58.1663853521 214.843319663


## Mathematical Versions

In [165]:
def radec_to_galactic_pos2(ra, dec, alpha_NGP=alpha_NGP, delta_NGP=delta_NGP, gc_GAN=gc_GAN):
    #Finds galactic coords (l, b) of an object given RA, DEC in degrees.       
    b = asin_deg(sin_deg(dec)*cos_deg(90.0-delta_NGP)-cos_deg(dec)*sin_deg(ra-(alpha_NGP+90.0))*sin_deg(90.0-delta_NGP))
    y = cos_deg(dec)*sin_deg(ra-(alpha_NGP+90.0))*cos_deg(90.0-delta_NGP)+sin_deg(dec)*sin_deg(90.0-delta_NGP)
    x = cos_deg(dec)*cos_deg(ra-(alpha_NGP+90.0))
    l = atan2_deg(y,x)+gc_GAN
    if l < 0:
        l += 360
    return {'l':l, 'b':b}

radec_to_galactic_pos2(test_ra, test_dec)

{'b': 67.231242804005689, 'l': 220.16912488386765}